# IN4080: obligatory assignment 4
 
The final mandatory assignment for IN4080 consists of two parts. The first is about the development of dialogue systems, and the second about machine translation.
You are required to get at least 12/20 points to pass. 

- We assume that you have read and are familiar with IFI’s requirements and guidelines for mandatory assignments, see [here](https://www.uio.no/english/studies/examinations/compulsory-activities/mn-ifi-mandatory.html) and [here](https://www.uio.no/english/studies/examinations/compulsory-activities/mn-ifi-guidelines.html).
- This is an individual assignment. You should not deliver joint submissions. 
- You may redeliver in Devilry before the deadline (__Sunday, November 7 at 23:59__).
- Only the last delivery will be read! If you deliver more than one file, put them into a zip-archive. You don't have to include in your delivery the data files already provided for this assignment. 
- Name your submission _your\_username\_in4080\_mandatory\_4_

Part 1 should be done on your local computer, as it relies on a speech interface that will not work on remote machines. For Part 2, using _Fox_ is preferable, at least for the fine-tuning task.

You should deliver a completed version of this Jupyter notebook, containing both your code and explanations about the steps you followed. We want to stress that simply submitting code is __not__ by itself sufficient to complete the assignment - we expect the notebook to also contain explanations of what you have implemented, along with motivations for the choices you made along the way. Preferably use whole sentences, and mathematical formulas if necessary. Explaining in your own words (using concepts we have covered through in the lectures) what you have implemented and reflecting on your solution is an important part of the learning process - take it seriously!

Regarding the use of LLMs (ChatGPT or similar): you are allowed to use them as 'sparring partner', for instance to clarify something you have not understood. However, you are __not__ allowed to use them to generate solutions (either in part or in full) to the assignment tasks. 


## Part 1 : Dialogue systems

Our objective in this part is to build a spoken conversational interface for a (simulated) elevator. 

### Basic setup

First, let's make sure that we have `ipywidgets` installed::

In [ ]:
%pip install ipywidgets

The code for the simulated elevator is provided below. The elevator is displayed using simple widgets (where the current floor is shown in green). 

In [1]:
import ipywidgets as widgets
from IPython.display import display
import asyncio

import time, random, string, threading
from typing import List, Tuple, Dict, Set
   
class BasicElevator:
    """Representation of an elevator. The elevator is simulated using a GUI"""
    
    def __init__(self, start_floor:int =1, nb_floors=10):
        """Initialised a new elevator, placed on the first floor"""
        
        # Current floor of the elevator
        self.cur_floor: int = start_floor
        self._moving: bool = False
        
        self._build_gui(nb_floors)

        # (Possibly empty) list of next floor stops to reach
        self.nextstops: List[int] = []

        display(self.gui)


    def move_to_floor(self, floor_number: int):
        """Move to a given floor (by adding it to a stack of floors to reach)"""
        
        if floor_number < 1 or floor_number > len(self.floors):
            raise RuntimeError("Floor number must be between 1 and %i"%len(self.floors))

        self.nextstops.append(floor_number)
        asyncio.create_task(self._trigger_movement())
        
        
    def stop(self):
        """Stops all movements of the elevator"""
        
        self.nextstops.clear()
        asyncio.create_task(self._trigger_movement())


    def _build_gui(self, nb_floors):
        """Creates the GUI for the elevator, with a status label and a visual representation
        of the floors, where the current floor is indicated in green."""

        # Displaying the current status of the elevator (still or going up or down)
        status_label = widgets.HTML("<b>Status</b>: ")
        self.status = widgets.Label("Still")
        status_box = widgets.HBox([status_label, self.status])

        # Displaying the floors on a vertical axis
        self.floors = []
        floor_layout = widgets.Layout(width='50px', height='30px', border='2px solid black',justify_content="center")
        for i in range(1, nb_floors+1):
            floor = widgets.Label(value=str(i), layout=floor_layout)
            floor.style = {"background":("white" if i!=self.cur_floor else "lightgreen")}
            self.floors.append(floor)

        # Create a vertical box container to hold the boxes
        vbox = widgets.VBox([status_box] + self.floors[::-1])
        self.gui = vbox

    async def _trigger_movement(self, speed=1.0):
        """Triggers the movement of the elevator towards the next stops in the list.
        The elevator moves one floor at a time, waiting 'speed' seconds between floors."""

        # If already moving, do nothing
        if self._moving:
            return
        
        self._moving = True
        while self.nextstops:
            floor_to_reach = self.nextstops[0]
            
            if self.cur_floor == floor_to_reach:
                self.nextstops.pop(0)
                continue
            
            if floor_to_reach > self.cur_floor:
                next_floor = self.cur_floor + 1
                self.status.value = "UP"
            else:
                next_floor = self.cur_floor - 1
                self.status.value = "DOWN"
            
            # Added sleep to simulate time taken to move between floors
            await asyncio.sleep(speed)

            # Update UI safely
            self.floors[self.cur_floor - 1].style.background = "white"
            self.floors[next_floor - 1].style.background = "lightgreen"
            self.cur_floor = next_floor
             
        self.status.value = "Still"
        self._moving = False

The elevator can be easily controlled through the functions `move_to_floor` and `stop`:

In [7]:
elevator = BasicElevator()
elevator.move_to_floor(5)


We will now make our elevator controllable through a speech interface instead of using function calls.

Before we begin, you need to install `pyaudio` (for audio processing), `whisper` (for speech recognition), and `pyttsx3` (for speech synthesis):
```bash
%pip install pyaudio openai-whisper pyttsx3
```

Audio processing in Python can be quite tricky, as it works differently from OS to OS. In particular, if you are running on Linux and the TTS is not working, install the following packages on your machine: `sudo apt update && sudo apt install espeak ffmpeg libespeak1`. If you are on MacOS and TTS is not working, try installing portaudio using Homebrew: `brew install portaudio`.

If you run into issues with `pyaudio`, try using `sounddevice` instead: ```bash pip install sounddevice```.

_If you still run into issues with the audio processing, please let us know (ideally on Discourse), we have a number of fixes that can help_.

## Speech interface

The `TalkingElevator` class below extends the basic simulated elevator with speech input and output. 

Upon clicking on the recording button, speech is recorded from the user's microphone, and continues until the stop button is clicked. The speech recognition engine `Whisper` from OpenAI is then employed to transcribe the spoken input (either on GPU, if you have a GPU on your machine, or on CPU). The transcription result is then sent to the `process_input` function, which is responsible for determining the system response. 

We are going to focus on implementing this `process_input` method. Note this system reaction to new user inputs may comprise both verbal responses (to be uttered by the system through the `_say_to_user` method) and physical actions (through the `move_to_floor` and `stop` methods).

In [8]:
import threading, time
import numpy as np
from typing import List, Literal, Tuple, Dict, Set
import whisper, pyttsx3
import warnings

class TalkingElevator(BasicElevator):
    """Extension of the simulated elevator with a speech interface"""
    
    def __init__(self, audio_backend:Literal["pyaudio","sounddevice"]="pyaudio"):
        """Initialises the Talking Elevator, loading the ASR model and setting up the GUI.
        The audio_backend parameter specifies which library to use for audio recording
        (either 'pyaudio' or 'sounddevice')."""

        self.audio_backend = audio_backend
        print("Loading ASR model", end="...", flush=True)
        self.asr_engine = whisper.load_model("small.en")
        print("Done")
        
        # Initializing the GUI
        BasicElevator.__init__(self)

        # Starts the dialogue
        self.dialogue_history = []
        self._say_to_user("Hi, what can I do for you today?")
    

    def process_input(self, user_input: str, conf_score:float=1.0):
        """Processes the (transcribed) user input, and respond appropriately 
        (through a verbal response and possibly also an action, such as moving floors)"""

        self._add_to_dialogue_history(user_input, speaker="user", conf_score=conf_score)

        # Dummy response. Should be replaced by the actual dialogue behaviour
        self._say_to_user("Sorry, I don't understand you, pal")

    
    def _say_to_user(self, system_response: str):
        """Say something back to the user, and add the dialogue turn to the history. The 
        synthesis is done using the pyttsx3 library."""

        self._add_to_dialogue_history(system_response, speaker="elevator")
        
        # Due to resource management issues, we initialize a new TTS engine for each utterance
        tts_engine = pyttsx3.init()  
        tts_engine.say(system_response)
        tts_engine.runAndWait()
        tts_engine.stop()
        del tts_engine


    def _add_to_dialogue_history(self, turn:str , speaker:str, conf_score:float=1.0):
        """Adds a new (user or system) turn to the dialogue history list, and displays it
         on the chat window displaying the turns"""

        self.dialogue_history.append({"speaker":speaker, "text":turn, 
                                      "conf_score":conf_score, "timesamp":time.time()})
        
        self.history_area.value += "&nbsp;<strong>%s</strong>:  %s"%(speaker.title(), turn)
        if conf_score < 1.0:
            self.history_area.value += " (%.2f)"%(conf_score)
        self.history_area.value += "<br>"
   
    
    def _build_gui(self, nb_floors):
        """GUI for the Talking elevator, comprising (beyond the simulated elevator from 
        BasicElevator) a chat window showing the dialogue turns, and buttons to record
        the user input. 
        The user should first click on the record button, then on stop when they have finished.
        Once the stop button is clicked, the audio is transcribed by Whisper, and finally 
        forwarded to the process_input function."""

        BasicElevator._build_gui(self, nb_floors)

        self.frames = []
        self.recording = False

        def record(chunk_size=1024):
            """Record audio chunks to a buffer."""
            self.recording = True

            frames = []
            if self.audio_backend=="pyaudio":
                import pyaudio
                p = pyaudio.PyAudio()
                stream = p.open(format=pyaudio.paInt16, channels=1, rate=16000, input=True, 
                                frames_per_buffer=chunk_size)
                while self.recording:
                    self.frames.append(stream.read(chunk_size))
                stream.close()
                p.terminate()

            elif self.audio_backend=="sounddevice":
                import sounddevice
                stream = sounddevice.InputStream(samplerate=16000, channels=1, blocksize=chunk_size)
                stream.start()
                while self.recording:
                    data, overflowed = stream.read(chunk_size)
                    if overflowed:
                        print("Overflow occurred!")
                    self.frames.append(data.copy())
                stream.stop()
                stream.close()

        def on_record_button_clicked(b):
            "Starts the recording"
            record_button.disabled=True
            stop_button.disabled=False
            audio_status.value = "Recording..."
            self.frames = []  # Clear previous recordings
            thread = threading.Thread(target=record)
            thread.start()

        def on_stop_button_clicked(b):
            "stops the recording, runs Whisper, and forward the result to process_input"
            self.recording = False
            record_button.disabled=False
            stop_button.disabled=True
            audio_status.value = "Transcribing..."
            if self.audio_backend=="pyaudio":
              audio_data = np.frombuffer(b"".join(self.frames), np.int16).astype(np.float32) * (1 / 32768.0)
            elif self.audio_backend=="sounddevice":
                audio_data = np.concatenate(self.frames, axis=0).flatten()
            output = self.asr_engine.transcribe(audio_data, fp16=False)
            audio_status.value = "Done transcribing."

            # We define the confidence score based on the log-probabilities
            conf_score = np.exp(np.mean([segment["avg_logprob"] for segment in output["segments"]]))
            # (and we push those up a bit, as the Whisper scores seem too low)
            conf_score = min(1, conf_score*1.2)

            # Finally, we process the input
            self.process_input(output["text"], conf_score)

        # The record and stop buttons
        record_button = widgets.Button(icon="microphone")
        stop_button = widgets.Button(icon="stop", disabled=True)
        audio_status = widgets.Label("")
        record_button.on_click(on_record_button_clicked)
        stop_button.on_click(on_stop_button_clicked)
        
        # The chat area
        self.history_area = widgets.HTML(layout=widgets.Layout(width="600px", height="300px", 
                                                               border='1px solid black', overflow='scroll'))

        # The right side of the GUI
        right_side = widgets.VBox([widgets.Label(""), self.history_area, widgets.HBox([record_button, stop_button, audio_status])])
        self.gui = widgets.HBox([self.gui, right_side])


Let's give it a try:

In [6]:
elevator = TalkingElevator(audio_backend="pyaudio")

Loading ASR model...Done




**Note**: The current implementation reloads the TTS and ASR models every time, which means that you may run into a "CUDA: out of memory" error if you reinitialise the `TalkingElevator` many times. If this happens, simply restart the Python kernel, which will clear the memory on both the CPU and the GPU. 


### Intent recognition

We wish our talking elevator to support the following functionalities:
 
- If the user express a wish to go to floor $X$ (where $X$ is an integer value between 1 and 10), the elevator should go to that floor. The interface should allow for several ways to express a given intent, such as "_Please go to the $X$-th floor_" or "_Floor $X$, please_".
- The user requests can also be relative, for instance "_Go one floor up_".
- The elevator should provide _grounding_ feedback to the user. For instance, it should respond "_Ok, going to the $X$-th floor_" after a user request to move to $X$.  
- The elevator should handle misunderstandings and uncertainties, e.g. by requesting the user to repeat, or asking the user to confirm if the intent is uncertain (say, when its confidence score is lower than 0.5). 
- The elevator should also allow the user to ask where the office of a given employee is located. For instance, the user could ask "_where is Erik Velldal's office?_", and the elevator would provide a response such as "_The office of Erik Velldal is on the 4th floor. Do you wish to go there?_".  We provide you with the office numbers of a small set of IFI employees in the `OFFICES` dictionary (see below).
- The elevator should also be able to inform the user about the current floor (such as replying to "_Which floor are we on?_" or "_Are we on the 5th floor?_"). 
- Finally, if the user asks the elevator to stop (or if the user says "_no_" after a grounding feedback "_Ok, going to floor $X$._"), the elevator should stop, and ask for clarification regarding the actual user intent. 

To implement this conversational behaviour, we will rely on a classical NLU-based approach in which we will recognise the user _intent_, and then determine a response based on the recognised intent(s). 

__Task 1.1__ (1 point): You first need to define a list of user intents that cover the kinds of user inputs you expect to observe in this talking elevator, such as `RequestMoveToFloor` or `Confirm`. This is a design question, and there is no obvious right or wrong answer. Define below the intents you want to cover, along with an explanation and a few examples of user inputs for each.


<!-- Provide here the list of intent classes you have defined, together with an explanation and a few examples -->

__Task 1.2__ (1 points): We wish to build a classifier any user input to a probability distribution over those intents, and start by creating a small, synthetic training set. Make a list of about 100 user utterances, each labelled with an intent defined above. You can "make up" those utterances yourself, or ask someone else to come with alternative formulations if you lack inspiration.

In [1]:
labelled_utterances =  [("I really like the IN4080 course", "OutOfCoverage"), #... 
                        ]

We will now train an intent classifier based on the labelled utterances you have defined. You can choose between the following approaches:
- you can use the [SetFit](https://github.com/huggingface/setfit) library to fine-tune a sentence-transformer model for classify your utterances. If you choose this option, install the `setfit` library  (`pip install setfit`) and read the [Setfit quickstart guide](https://huggingface.co/docs/setfit/quickstart) to find out how to use it.
- Alternatively (if SetFit is too slow on your machine or you have technical problems with it), you can also train a simple classifier based on sentence embeddings from `sentence-transformers` (install it using `pip install sentence-transformers`). You can e.g. use a k-NN classifier or a logistic regression model from `sklearn` on top of the sentence embeddings. Or even use bag-of-words features if you prefer.

__Task 1.3__ (2 points): Train your intent classifier based on `labelled_utterances`. Once you are done, fill the implementation of the `IntentClassifier` class below.

In [ ]:
class IntentClassifier:

    def __init__(self, init_args):
        """Initialises the intent classifier, for instance by loading a fine-tuned 
        SetFit model. You should replace init_args by whatever arguments you need."""

        raise NotImplementedError("Must be implemented")


    def get_intent_distrib(self, utterance:str) -> Dict[str, float]:
        """Applies the trained model on a new utterance. The method should return a
        dictionary that maps each possible intent category to a probability."""

        raise NotImplementedError("Must be implemented")


You can then test your classifier as follows:

In [ ]:
classifier = IntentClassifier()
classifier.get_intent_distrib("Take me to the fifth floor, please")

Since we don't have any test data, we cannot really conduct an evaluation of the classification performance, but this step would be of course strongly advised when developing a real system. 

### Slot filling

In addition to the intents themselves, we also wish to detect some slots, such as floor numbers or person names. For this step, we will not use a data-driven model, but rather rely on an old-fashioned, rule-based approach:
- For floor numbers, we will rely on string matching (with regular expressions or basic string search) that detect patterns such as "X floor" (where X is [first,second, third, fourth, fifth, sixth, seventh, eighth, ninth, tenth]) or "floor X" (where X is between 1 and 10).
- For person names, we have a predefined list of person names to detect (employees at IFI), and we should simply search for their occurrence in the user input. The simplest implementation is to just for look for exact occurrences. However, since speech recognition will often struggle to recognize foreign person names, an even better approach would be to search for names that are phonetically close (you can use the `jellyfish` library for this).

The results of the slot filling should be a dictionary mapping slot names to a canonical form of the slot value. For instance, if the utterance contains the expression "ninth floor", the resulting slot dictionary should be `{"floor_number":9}`. Similarly, the `employee_name` slot should be a name present in `OFFICES` dictionary. 

__Task 1.4__ (2 points): Implement the method `fill_slots` that will detect the occurrence of those slots in the user input.<br>
(+ 1 bonus point if you implement a fuzzy matching strategy to find person names that are phonetically close)

In [21]:

# Floor numbers for a subset of the IFI employees
OFFICES = {'Adín Ramírez Rivera': 4, 'Andreas Austeng': 4, 'Anne H Schistad Solberg': 4, 
           'Arild Torolv Søetorp Waaler': 9, 'Audun Jøsang': 9, 'Birthe Soppe': 4, 'Carsten Griwodz': 4,
           'Dag Sjøberg': 9, 'Dag Trygve Eckhoff Wisland': 5, 'Einar Broch Johnsen': 8, 
           'Eric Bartley Jul': 10, 'Erik Velldal': 4, 'Henrik Skaug Sætra': 7, 'Ingrid Chieh Yu': 8,
           'Jørn Anders Braa': 6, 'Kristin Bråthen': 4, 'Kyrre Glette': 4, 'Lars Groth': 6, 
           'Lilja Øvrelid': 4, 'Maja Van Der Velden': 7, 'Martin Giese': 9, 'Michael Welzl': 5, 
           'Miria Grisot': 6, 'Nils Gruschka': 9, 'Olaf Owe': 9, 'Ole Christian Lingjærde': 4, 
           'Ole Hanseth': 6, 'Paulo Ferreira': 10, 'Philipp Dominik Häfliger': 5, 'Philipp Häfliger': 5, 
           'Roman Vitenberg': 4, 'Silvia Lizeth Tapia Tarifa': 8, 'Stephan Oepen': 4, 
           'Sundeep Sahay': 6, 'Thomas Peter Plagemann': 4, 'Tone Bratteteig': 7, 'Torbjørn Rognes': 8, 
           'Truls Erikson': 6, 'Viktoria Stray': 10, 'Yngvar Berg': 5, 'Yves Scherrer': 4, 
           'Özgü Mira Alay-Erduran': 4}

def fill_slots(user_input:str) -> Dict[str,str]:
    """Extracts the set of slots detected in the user inputs. More precisely, the method
    should detect both floor numbers and person names, and return a dictionary mapping slot 
    names (in this case either `floor_number` or `employee_name`) to its corresponding
    value, in canonical form (integer for the floor number, string for the employee name)"""

    raise NotImplementedError()


### Response selection

The next step is to implement the response selection mechanism. The response will depend on various factors:
- the inferred user intents from the user utterance
- the detected slot values in the user utterance (if any)
- the current floor
- the list of next floor stops that are yet to be reached
- the dialogue history (as a list of dialogue turns).

The response may consist of verbal responses (enacted by calls to `_say_to_user`) but also physical actions, represented by calls to either `move_to_floor` or `stop`. 

__Task 1.5__ (3 points): Implement the method `_respond`, which is responsible for selecting and executing those responses. The responses should satisfy the aforementioned conversational criteria (provide grounding feedback, use confirmations and clarification requests etc.). This method will consist in practice of many _if...then...else_ blocks. 

In [ ]:
def _respond(self, intent_distrib: Dict[str, float], slots: Dict[str,str]) :
    """Given a probability distribution over possible intents, an a (possibly empty) list
    of detected slots in the user input, decide how to react. The method should lead
    to calls to both physical actions (move_to_floor, stop) and dialogue responses 
    (via _say_to_user)."""

    raise NotImplementedError("")

setattr(TalkingElevator, "_respond", _respond)



### Putting it all together

The last step is to implement the `process_input` method in the `TalkingElevator` class. The method should rely on the intent recognition, slot filling and response selection mechanism (which you have implemented in the previous steps) to react to a given user input.

**Task 1.6** (1 point): Implement the `process_input` method:

In [ ]:
def process_input(self, user_input: str, conf_score:float=1.0):
    """Processes the (transcribed) user input, and respond appropriately 
    (through a verbal response and possibly also an action, such as moving floors).
    The method should rely on the intent classifier, slot-filling function, and
    response selection function."""

    self._add_to_dialogue_history(user_input, speaker="user", conf_score=conf_score)
    raise NotImplementedError()

setattr(TalkingElevator, "process_input", process_input)

We are now ready to test our talking elevator: 


In [ ]:
elevator = TalkingElevator()

Your talking elevator will mostly likely not function properly right from the start. Identify what works and what doesn't and correct the code you have developed in Tasks 1.1 - 1.6 until your system meets the specifications we have outlined. 

## Part 2 : Machine translation

In this part, we evaluate a pre-trained machine translation model on data from the Lord of the Rings movies and fine-tune it to improve the translation quality.

### Data

We provide you with two files, `lotr.detok.de` and `lotr.detok.en`, containing German and English movie subtitles. These two files constitute a so-called _parallel corpus_, i.e. each sentence/line in German corresponds to a sentence/line in English. The two files have the same number of lines and the German sentence on line $i$ corresponds to the English sentence on line $i$. The subtitles are extracted from the [OpenSubtitles-2018](https://opus.nlpl.eu/OpenSubtitles/corpus/version/OpenSubtitles) corpus.

Here are the first ten lines of the two files:

<style scoped>
table {
  font-size: 12px;
}
</style>
| Nb  | German (`lotr.detok.de`)         | English (`lotr.detok.en`)      |
|---|----------------------------------|--------------------------------|
| 1 | Die Welt ist im Wandel. | The world is changed.   |
| 2 | Ich spüre es im Wasser. | I feel it in the water. |
| 3 | Ich spüre es in der Erde. | I feel it in the earth. |
| 4 | Ich rieche es in der Luft. | I smell it in the air. |
| 5 | Vieles, was einst war, ist verloren, da niemand mehr lebt, der sich erinnert. | Much that once was is lost. For none now live who remember it. |
| 6 | Es begann mit dem Schmieden der Großen Ringe. | It began with the forging of the Great Rings. |
| 7 | 3 wurden den Elben gegeben, den unsterblichen, weisesten und reinsten aller Wesen. | Three were given to the Elves: Immortal, wisest and fairest of all beings. |
| 8 | 7 den Zwergenherrschern, großen Bergleuten und Handwerkern in ihren Hallen aus Stein. | Seven to the Dwarf-lords: Great miners and craftsmen of the mountain halls. |
| 9 | Und 9... 9 Ringe wurden den Menschen geschenkt, die vor allem anderen nach Macht streben. | And nine nine rings were gifted to the race of Men who, above all else, desire power. |
| 10 | Denn diese Ringe bargen die Kraft und den Willen, jedes Volk zu leiten. | For within these rings was bound the strength and will to govern each race. |


In [20]:
%pip install unbabel-comet==2.2.2

Defaulting to user installation because normal site-packages is not writeable
  Using cached protobuf-4.25.8-cp37-abi3-manylinux2014_x86_64.whl.metadata (541 bytes)
Using cached protobuf-4.25.8-cp37-abi3-manylinux2014_x86_64.whl (294 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorboard 2.15.1 requires protobuf<4.24,>=3.19.6, but you have protobuf 4.25.8 which is incompatible.

[notice] A new release of pip is available: 23.3.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Getting started

We will a pretrained machine translation model for German-to-English translation. The model is available on the HuggingFace model hub and can be used with the `transformers` library.

Let us first make sure that all required modules are installed:

In [1]:
%pip install torch transformers accelerate evaluate sacrebleu sacremoses sentencepiece unbabel-comet

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.8/375.8 kB 5.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.0/91.0 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.4/101.4 kB 1.7 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 20.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 832.4/832.4 kB 18.2 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 529.7/529.7 kB 12.6 MB/s eta 0:00:0000:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependen

The bilingual model is called [`opus-mt-de-en`](https://huggingface.co/Helsinki-NLP/opus-mt-de-en) and has been trained by the Helsinki-NLP group. Like (almost) all HuggingFace models, it consists of a _tokenizer_ and the _sequence-to-sequence model_ properly speaking. We need to load both separately:

In [2]:
import transformers

tokenizer = transformers.AutoTokenizer.from_pretrained("helsinki-nlp/opus-mt-de-en")
translator = transformers.AutoModelForSeq2SeqLM.from_pretrained("helsinki-nlp/opus-mt-de-en")

# Change "cuda" to "cpu" if you're running on a machine without GPU
device = "cuda"
translator = translator.to(device)

/fp/projects01/ec30/software/easybuild/software/nlpl-pytorch/2.1.2-foss-2022b-cuda-12.0.0-Python-3.10.8/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


The `transformers` library will automatically download the models from the HuggingFace hub the first time you run this cell, so it may take a bit longer.

Let's take the first two German sentences, tokenize them, and translate them to English:

In [3]:
tokens = tokenizer(["Die Welt ist im Wandel.", "Ich spüre es im Wasser."], return_tensors="pt", padding=True)
print(tokens)

{'input_ids': tensor([[   55,   401,    29,    49,  9012,     3,     0, 58100, 58100],
        [  105,  2768,  1691,    18,    65,    49,   672,     3,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [4]:
outputs = translator.generate(**tokens.to(device), max_new_tokens=50)
print(outputs)

tensor([[58100,    36,   360,    19,  7315,     3,     0, 58100, 58100, 58100],
        [58100,    38,    85,  1595,    56,     5,     4,   616,     3,     0]],
       device='cuda:0')


__Task 2.1__ (1 point):
- What do the numbers in the `input_ids` represent?
- What is the effect of `padding=True`? How would the data look like if padding was disabled?
- What does `max_new_tokens` do? Why do you think it is important to set this parameter?

Shu_explanation:
- The number in the `input_ids` represents that after the tokenization, the vocab_index for each subword in the sentence(not necessarily a whole word,because the tokenizer uses subword segmentation).
- Transformers require all sequences in a batch to have the same length. Setting `padding=True` ensures this shape alignment by automatically adding <pad> tokens to shorter sentences so they match the length of the longest one in the batch. If padding was disabled, then the length for each sentence would be misaligned. Sentences of different lengths could not be put into a single tensor, and the tokenizer would raise an error such as: `ValueError`: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`input_ids` in this case) have excessive nesting (inputs type `list` where type `int` is expected).
- `max_new_tokens` defines the maximum number of tokens the model is allowed to generate during decoding. It is important to set a limit to avoild the model to generate excessively long or infinite outputs.

We can get actual words by running the output through the `batch_decode` function of the tokenizer:

In [5]:
translations = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(translations)

['The world is changing.', 'I can feel it in the water.']


__Note:__ We assume that you will run the translations from German to English. If you would like to work on the opposite translation direction (and feel comfortable evaluating the German output), you are welcome to do so. The corresponding bilingual model is called `opus-mt-en-de`.

### Data splitting

Before we move on, we need to split our data. We will evaluate different models and for that we'll need test data. We will also fine-tune a model, and for that we'll need training data. The entire Lord of the Rings dataset has 9640 lines.

__Task 2.2__ (1 point): Split the dataset in such a way that the **last** 1000 lines are used for testing and the remaining lines (8640) for training. Save the data under the following filenames: `lotr.train.de, lotr.train.en, lotr.test.de, lotr.test.en`. You can use Python code or other tools to perform the splitting.

In [6]:
#load lotr.detok.de
with open('lotr.detok.de', 'r', encoding='utf-8-sig') as f:
    text_de = f.readlines()
print(len(text_de),type(text_de))   

9640 <class 'list'>


In [7]:
#load lotr.entok.en
with open('lotr.detok.en', 'r', encoding='utf-8-sig') as f:
    text_en = f.readlines()  
len(text_en)   

9640

In [8]:
# construct training set and testing set for de, and save to lotr.train.de,lotr.test.de
train_de = text_de[:-1000]
test_de = text_de[-1000:]
with open('lotr.train.de', 'w', encoding='utf-8') as f:
    f.write(''.join(train_de))
with open('lotr.test.de', 'w', encoding='utf-8') as f:
    f.write(''.join(test_de))

In [9]:
# construct training set and testing set for en,and save to lotr.train.en,lotr.test.en
train_en = text_en[:-1000]
test_en = text_en[-1000:]
with open('lotr.train.en', 'w', encoding='utf-8') as f:
    f.write(''.join(train_en))
with open('lotr.test.en', 'w', encoding='utf-8') as f:
    f.write(''.join(test_en))

__Task 2.3__ (1 point): What are potential risks and drawbacks of splitting the dataset in this way? 

Shu_explanation:
Splitting the dataset by simply taking the last 1000 lines for testing introduces several potential risks:
- Sequential bias / lack of randomness:
  Since the data are not randomly selected, the test set may come from a specific part of the corpus that differs stylistically or semantically from the training set.
- Limited test size
  Using only 1000 examples (around 10% of the data) might provide too small a sample to reliably estimate generalization performance.

Now we are ready to translate the test set with our model.

__Task 2.4__ (2 points): Create a function that loads the entire `lotr.test.de` file, translates each line with the `opus-mt-de-en` model and writes its output to a new file, one sentence per line.

The easiest way to do this is to just load the entire test file into a list, tokenize and translate it, but the test set may be too large to fit on GPU memory, or it might be inefficient and slow if you use a CPU. A better alternative is to split the data into batches of 50-100 sentences and send each batch separately to the translator.

In [10]:
import numpy as np
nums = np.random.randint(0, 100, size=100).tolist()
batch_size=50
for i in range(0,len(nums),batch_size):
    batch = nums[i : i + batch_size]
    print(batch)

[3, 74, 59, 68, 33, 80, 71, 94, 80, 94, 17, 28, 35, 77, 45, 61, 83, 55, 58, 41, 94, 60, 6, 94, 66, 86, 40, 58, 90, 20, 7, 96, 6, 34, 39, 44, 77, 92, 24, 77, 19, 36, 77, 35, 96, 72, 14, 8, 29, 72]
[58, 78, 85, 63, 25, 42, 9, 0, 35, 6, 77, 6, 0, 28, 85, 22, 24, 64, 42, 22, 33, 55, 53, 46, 31, 65, 42, 90, 85, 51, 80, 78, 54, 86, 18, 54, 24, 83, 75, 79, 24, 63, 35, 18, 70, 97, 79, 26, 50, 98]


Shu_explanation:
test batch_size using 100 int in a list.

In [11]:
#load lotr.test.de file
with open('lotr.test.de','r',encoding='utf-8') as f:
        test_de = f.readlines()
test_de[:5]

['Hier ist nichts.\n',
 'Sucht weiter!\n',
 'Dieser Stein könnte überall sein.\n',
 'Der Arkenstein liegt in diesen Hallen.\n',
 '- Findet ihn!\n']

In [12]:
# strip lines
test_de_data=[]
for line in test_de[:5]:
    line = line.strip()
    test_de_data.append(line)
test_de_data

['Hier ist nichts.',
 'Sucht weiter!',
 'Dieser Stein könnte überall sein.',
 'Der Arkenstein liegt in diesen Hallen.',
 '- Findet ihn!']

In [13]:
# test translate in batch_size=2
for i in range(0,5,2):
    batch = test_de_data[i:i+2]
    tokens = tokenizer(batch, return_tensors="pt", padding=True)
    outputs = translator.generate(**tokens.to(device), max_new_tokens=50)
    translations = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    print(translations)

["There's nothing here.", 'Keep looking!']
['This stone could be anywhere.', 'The Arkenstein is located in these halls.']
['- Find him!']


In [14]:
for i in range(0,5,2):
    batch = test_de_data[i:i+2]
    tokens = tokenizer(batch, return_tensors="pt", padding=True)
    outputs = translator.generate(**tokens.to(device), max_new_tokens=50)
    translations = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    with open('test_de.en','a',encoding='utf-8') as f:
        f.write('\n'.join(translations)+ '\n')

Shu_explation:
We need to use 'a' append instead of 'w' write. Also, we need to add '\n' at the end of each batch. Use a small batch of 5 sentences and batch_size=2 to test everything is ok. Then we can put it all together. 

In [15]:
def translate(input_file, translation_file, tokenizer, translator, batch_size=100):
    """Translate an input file line by line using the loaded tokenizer and translator,
    and write the translations to output_file."""
    test_data = []
    with open(input_file,'r',encoding='utf-8') as f:
        test_data = [line.strip() for line in f.readlines()]
    for i in range(0,len(test_data),batch_size):
        batch = test_data[i: i + batch_size]
        tokens = tokenizer(batch, return_tensors="pt", padding=True)
        outputs = translator.generate(**tokens.to(device), max_new_tokens=50)
        translations = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        with open(translation_file,'a',encoding='utf-8') as f:
            f.write('\n'.join(translations)+'\n')
        
translate("lotr.test.de", "lotr.output_opus.en", tokenizer, translator)

Before moving on, open the output file and check that the translations look ok. In particular, the file should contain the expected number of lines and output should be in the expected language (English or German, depending on the chosen direction).

__Task 2.5__ (1 point): Open both the output file and the reference translations (`lotr.test.en` if translating from German to English) and compare the first 20 lines. How would you rate the translations of the OPUS system on a scale from 1 (incomprehensible and/or completely different meaning) to 5 (grammatically correct and meaning fully preserved)? Justify your answer.

In [16]:
import pandas as pd

with open ('lotr.output_opus.en','r', encoding='utf-8') as f:
    output = [line.strip() for line in f.readlines()][:20]

with open('lotr.test.en','r',encoding='utf-8') as f:
    test_en = [line.strip() for line in f.readlines()][:20]
df = pd.DataFrame({
    'target':output,
    'reference': test_en
})
df

,target,reference
0,There's nothing here.,Nothing here.
1,Keep looking!,Keep searching!
2,This stone could be anywhere.,That Jewel could be anywhere.
3,The Arkenstein is located in these halls.,The Arkenstone is in these halls.
4,- Find him!,- Find it!
5,You heard me.,You heard him.
6,- Keep looking.,- Keep looking.
7,All of you!,All of you!
8,No one rests until he's found.,No one rests until... it is found.
9,It's almost tempting me to leave it to you.,I am almost tempted... to let you take it.


Shu_explanation:
Here we conduct a reference-based evaluation. I will evaluate the output in two dimensions: 
- Adequacy: Does the output convey the same meaning as the input?
  I would rate 3 out of 5. Because some sentences don't convey the same meaning as the input. For example, translate "him" to "me". or "it" to "he". 
- Fluency: Is the output grammatically correct and idiomatic?
  I would rate the target lanuage translation 5 out of 5, since the target-language sentences are grammatically correct and sound natural.

### Evaluation

We can now evaluate the quality of our translations. In a first step, we perform _reference-based surface-level evaluation_  using the popular BLEU score. We can do that with the `sacrebleu` module. Below is a slightly reformatted example taken from the [SacreBLEU documentation](https://github.com/mjpost/sacrebleu/tree/master?tab=readme-ov-file#using-sacrebleu-from-python):

In [17]:
from sacrebleu.metrics import BLEU

reference = ['The dog bit the man.', 'It was not unexpected.', 'The man bit him first.']
hypothesis = ['The dog bit the man.', "It wasn't surprising.", 'The man had just bitten him.']

bleu_scorer = BLEU()
# BLEU can deal with multiple references per sentence, but here we only have one, so we just enclose it in another set of brackets:
score = bleu_scorer.corpus_score(hypothesis, [reference])
print(score)

BLEU = 45.07 70.6/42.9/36.4/37.5 (BP = 1.000 ratio = 1.000 hyp_len = 17 ref_len = 17)


__Task 2.6__ (1 point): Load both the system output and the reference of your test set and compute the corpus-level BLEU score. Also compute the corpus-level chrF score. Which of the scores is higher?

In [18]:
from sacrebleu.metrics import BLEU, CHRF

def evaluate_bleu(hypothesis_file, reference_file):
    
    with open (hypothesis_file,'r', encoding='utf-8') as f:
        output = [line.strip() for line in f.readlines()]
    with open(reference_file,'r',encoding='utf-8') as f:
        references = [line.strip() for line in f.readlines()]

    # initialize bleu and chrF
    bleu = BLEU()
    chrf = CHRF()

    #calculate bleu
    bleu_score = bleu.corpus_score(output, [references])
    print(f"Corpus BLEU: {bleu_score}")

    #calculate chrF
    chrf_score = chrf.corpus_score(output, [references])
    print(f"Corpus chrF: {chrf_score}")
    
evaluate_bleu("lotr.output_opus.en", "lotr.test.en")

Corpus BLEU: BLEU = 29.03 63.3/38.3/25.2/16.2 (BP = 0.920 ratio = 0.923 hyp_len = 6203 ref_len = 6721)
Corpus chrF: chrF2 = 51.05


Shu_explanation:
- BLEU(BiLingual Evaluation Understudy), is calucated by computing n-gram overlap (1-gram/2-gram/3-gram/4-gram precision: 63.3/38.3/25.2/16.2) between system output and reference and take the geometric mean of overlap precisions. The result show that the overall BLEU score is 29.03. BP represents brevity penalty, BP= 0.920, which means the output has almost the same length as the reference, only a little bit shorter.
- chrF(character-level score) breaks down words into character n-grams, which give partial credits for matching stems without requiring a stemmer / lemmatizer.
- Although both BLEU and chrF measure translation quality, their scales are not directly comparable. BLEU is a precision-based metric operating on word-level n-gram matches, while chrF is an F1-based metric operating on character n-gram matches.

Besides string-based metrics, neural metrics have become increasingly popular lately, since they have been shown to correlate better with human judgements. The most popular neural metric is called COMET and it can be used with the HuggingFace `evaluate` package. The example below is from the [documentation](https://huggingface.co/spaces/evaluate-metric/comet/blob/main/README.md):

In [21]:
import evaluate

comet_metric = evaluate.load('comet')
src = ["Dem Feuer konnte Einhalt geboten werden", "Schulen und Kindergärten wurden eröffnet."]
hyp = ["The fire could be stopped", "Schools and kindergartens were open"]
ref = ["They were able to control the fire.", "Schools and kindergartens opened"]
comet_score = comet_metric.compute(predictions=hyp, references=ref, sources=src)
print(comet_score)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.5.5. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../.cache/huggingface/hub/models--Unbabel--wmt22-comet-da/snapshots/2760a223ac957f30acfb18c8aa649b01cf1d75f2/checkpoints/model.ckpt`
Encoder model frozen.
/fp/homes01/u01/ec-shuwa/.local/lib/python3.10/site-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']
/fp/homes01/u01/ec-shuwa/.local/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /cluster/software/EL9/easybuild/software/jupyter-ser ...
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://p

{'mean_score': 0.905142068862915, 'scores': [0.8385583758354187, 0.9717257618904114]}


__Task 2.7__ (1 point): Adapt this code to evaluate the output of the OPUS model. Note that COMET also requires the source text.

In [25]:
def evaluate_comet(hypothesis_file, reference_file, source_file):
    
    comet_metric = evaluate.load('comet')

    #load and create src list
    with open (source_file,'r', encoding='utf-8') as f:
        src = [line.strip() for line in f.readlines()]
        
    #load and create hyp list        
    with open (hypothesis_file,'r', encoding='utf-8') as f:
        output = [line.strip() for line in f.readlines()]

    #load and create reference list 
    with open(reference_file,'r',encoding='utf-8') as f:
        references = [line.strip() for line in f.readlines()]   

    comet_score = comet_metric.compute(predictions=output, references=references, sources=src)

    return comet_score['mean_score']

evaluate_comet('lotr.output_opus.en', 'lotr.test.en', 'lotr.test.de')


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.5.5. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../.cache/huggingface/hub/models--Unbabel--wmt22-comet-da/snapshots/2760a223ac957f30acfb18c8aa649b01cf1d75f2/checkpoints/model.ckpt`
Encoder model frozen.
/fp/homes01/u01/ec-shuwa/.local/lib/python3.10/site-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']
/fp/homes01/u01/ec-shuwa/.local/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /cluster/software/EL9/easybuild/software/jupyter-ser ...
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://p

0.8517506766319275

Shu_explanation:
- Unlike BLEU and chrF, which evaluate the results at the corpus level, COMET evaluates translations at the sentence level and then reports the mean_score as the average over all sentences.
- I chose to present only the mean_score here to improve readability.

### Fine-tuning

Let us see now if we can further improve the translation quality. We still haven't used the training set after all...

Fine-tuning a translation model with the `transformers` library is a bit convoluted. You need the following ingredients:
- A `Seq2SeqTrainer` object, which defines the initial model and its tokenizer, the training data, and the configuration parameters (as a `Seq2SeqTrainingArguments` object). The training process starts with the `train()` method.
- A `Seq2SeqTrainingArguments` object, which contains the configuration parameters, such as the number of training epochs, the path for saving the fine-tuned model, the learning rate etc.
- A `DataCollatorForSeq2Seq` object that takes care of splitting the training data into batches of appropriate size.
- A `DatasetDict` object containing the tokenized training data. Typically, the untokenized data is loaded into a `DatasetDict` object, and the tokenization function is applied to everything inside this `DatasetDict` using the `map()` function.

__Task 2.8__ (1 point): The code in the box below shows a working example using the pretrained OPUS model, but is limited to two sentence pairs. Complete the code to load the entire training data.

In [26]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import DataCollatorForSeq2Seq
from datasets import Dataset, DatasetDict

max_length = 100

# ds = Dataset.from_dict({
#     "src_text": ["Die Welt ist im Wandel.", "Ich spüre es im Wasser."],
#     "tgt_text": ["The world is changed.", "I feel it in the water."]
# })
# data = DatasetDict({"train": ds})

#load and create src list
with open ('lotr.train.de','r', encoding='utf-8') as f:
    src = [line.strip() for line in f.readlines()]

#target and create target_text list
with open ('lotr.train.en','r', encoding='utf-8') as f:
    target_text = [line.strip() for line in f.readlines()]

ds = Dataset.from_dict({
    "src_text": src,
    "tgt_text": target_text
})
data = DatasetDict({"train": ds})

def preprocess_function(examples):
    model_inputs = tokenizer(examples["src_text"], text_target=examples["tgt_text"], max_length=max_length, truncation=True)
    return model_inputs

tokenized_datasets = data.map(preprocess_function, batched=True)
print(tokenized_datasets)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=translator)

args = Seq2SeqTrainingArguments(
    output_dir="opus-mt-de-en-lotr",
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True
)

trainer = Seq2SeqTrainer(
    translator,
    args,
    train_dataset=tokenized_datasets["train"],
    data_collator=data_collator,
    tokenizer=tokenizer
)

trainer.train()

Map:   0%|          | 0/8640 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['src_text', 'tgt_text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 8640
    })
})


/fp/projects01/ec30/software/easybuild/software/nlpl-transformers/4.47.1-foss-2022b-Python-3.10.8/lib/python3.10/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/localscratch/2091964/ipykernel_3446632/387673564.py:48: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Step,Training Loss
500,1.183100


/fp/projects01/ec30/software/easybuild/software/nlpl-transformers/4.47.1-foss-2022b-Python-3.10.8/lib/python3.10/site-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[58100]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=810, training_loss=1.1150821261935764, metrics={'train_runtime': 40.5623, 'train_samples_per_second': 639.017, 'train_steps_per_second': 19.969, 'total_flos': 160830792400896.0, 'train_loss': 1.1150821261935764, 'epoch': 3.0})

Shu_explanation:
- The fine-tuning process completed 810 training steps over 3 epochs, with a final training loss of 1.12().
- The model trained efficiently, processing approximately 639 samples per second, and the total runtime was around 40 seconds.
- A training loss around 1.1 is considered a good result for neural machine translation models (values below 2.0 generally indicate that the model has learned meaningful patterns rather than producing random translations).
- This suggests that the model successfully adapted to the training data and improved its translation quality.

The model was fine-tuned for three epochs, and you should have three checkpoints in the `opus-mt-de-en-lotr` directory.

__Task 2.9__ (1 point): Choose one of the checkpoints and use it to translate the test set. Evaluate the test set with BLEU, chrF and COMET. Note that locally saved model files (and tokenizers) can be loaded in the same way as models from the HuggingFace hub, e.g. with the following command: `transformers.AutoModelForSeq2SeqLM.from_pretrained("opus-mt-de-en-lotr/checkpoint-810")`

Did fine-tuning help? Did fine-tuning help? Have a look at the first rows of the files. Do you agree with the metrics?

In [30]:
import transformers

model_path = "./opus-mt-de-en-lotr/checkpoint-810"

tokenizer = transformers.AutoTokenizer.from_pretrained(model_path)
translator = transformers.AutoModelForSeq2SeqLM.from_pretrained(model_path)

# Change "cuda" to "cpu" if you're running on a machine without GPU
device = "cuda"
translator = translator.to(device)
translate("lotr.test.de", "lotr.output_finetuned.en", tokenizer, translator)

In [31]:
#evaluate with new output "lotr.output_finetuned.en"
evaluate_comet('lotr.output_finetuned.en', 'lotr.test.en', 'lotr.test.de')

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.5.5. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../.cache/huggingface/hub/models--Unbabel--wmt22-comet-da/snapshots/2760a223ac957f30acfb18c8aa649b01cf1d75f2/checkpoints/model.ckpt`
Encoder model frozen.
/fp/homes01/u01/ec-shuwa/.local/lib/python3.10/site-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']
/fp/homes01/u01/ec-shuwa/.local/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /cluster/software/EL9/easybuild/software/jupyter-ser ...
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://p

0.8645248173177242

Shu_explanation:
Using the COMET metric, the pretrained OPUS model achieved a mean score of 0.8518, while the fine-tuned model (checkpoint-810) reached 0.8645.
This represents a small but consistent improvement (+0.013), suggesting that fine-tuning helped the model adapt better to the domain.
Since COMET captures both adequacy and fluency using contextual embeddings, the higher score indicates improved semantic fidelity rather than superficial token overlap.

In [32]:
import pandas as pd

with open ('lotr.output_opus.en','r', encoding='utf-8') as f:
    output = [line.strip() for line in f.readlines()][:20]

with open ('lotr.output_finetuned.en','r', encoding='utf-8') as f:
    finetuned = [line.strip() for line in f.readlines()][:20]

with open('lotr.test.en','r',encoding='utf-8') as f:
    test_en = [line.strip() for line in f.readlines()][:20]
    
df1 = pd.DataFrame({
    'opus':output,
    'finetuned': finetuned ,
    'reference':test_en
})
df1

,opus,finetuned,reference
0,There's nothing here.,There's nothing here.,Nothing here.
1,Keep looking!,Keep looking!,Keep searching!
2,This stone could be anywhere.,This stone could be anywhere.,That Jewel could be anywhere.
3,The Arkenstein is located in these halls.,The Arkenstone lies in these halls.,The Arkenstone is in these halls.
4,- Find him!,- Find him!,- Find it!
5,You heard me.,You heard me.,You heard him.
6,- Keep looking.,- Keep looking.,- Keep looking.
7,All of you!,All of you!,All of you!
8,No one rests until he's found.,No one rests until it is found.,No one rests until... it is found.
9,It's almost tempting me to leave it to you.,It's almost tempting me to let you have it.,I am almost tempted... to let you take it.


Shu_explanation:
- The COMET result shows that after fine tuning, there is only a slight improvement in output.
- We observed the first 20 sentences and found that:
  Line8, the finetuned output capture it correctly while opus mis-intepret to him.
  Line14 improved the output by capture "only".